# 🎬 Federated API Governance — MCP + A2A Discovery Demo

## End-to-end demo: MCP servers, A2A agents, and federated API Center

This notebook demonstrates how Azure API Management + Azure API Center enable **federated governance** across distributed API environments — covering **MCP servers**, **A2A agents**, and **REST APIs** in a single unified catalog.

### Architecture
- **RG 2** — Pre-deployed REST→MCP conversion (Weather, Catalog, Order) + remote MCPs
- **RG 3** — FastMCP containers (Weather, Catalog, Order, Calculator) deployed from code  
- **RG 4** — A2A agents (Outline, Title, Summary) + REST APIs + 9 remote MCPs
- **RG 1** — **Central API Center** aggregating all APIs from RG 2, 3, 4

### Demo story
| Part | Topic | Description |
|------|-------|-------------|
| 1-3 | **MCP Servers** | Explore existing APIs, deploy FastMCP containers, test |
| 4-5 | **Federation** | Deploy central APIC, aggregate from RG 2 + RG 3 |
| 6-7 | **MCP Discovery** | Unified catalog, dynamic invoke, cross-env agent workflow |
| 8-9 | **A2A Agents** | Explore A2A agents, aggregate to central, agent card discovery |
| 10 | **Unified Governance** | Full catalog: REST + MCP + A2A, metrics summary |

### Prerequisites
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and signed in
- [Python 3.12+](https://www.python.org/) with the project requirements installed
- An Azure subscription with **Contributor** access

---
## Part 1 — Explore Existing Distributed APIs (RG 2)
---

> RG 2 (`rg-lab-mcp-demo-1-1`) is **already deployed** — it has 4 REST APIs converted to MCP servers + remote MCPs registered in API Center.

### 0️⃣ Initialize Variables

In [ ]:
import os, sys, json, requests
sys.path.insert(1, '../../shared')
import utils

# ── RG 2: Existing mixed REST→MCP deployment ──
rg2_name = "rg-lab-mcp-demo-1-1"
rg2_apic = "apic-demo-1-xbilmxrmx74wu"
rg2_apim = "apim-mcp-demo-1"

# ── RG 3: New FastMCP containers deployment ──
rg3_name = "rg-lab-mcp-containers"
rg3_location = "uksouth"
rg3_apim_name = "apim-2-fastmcp"
rg3_apic_prefix = "apic-2-fastmcp"
rg3_deployment_name = "fastmcp-containers"

# ── RG 4: A2A agents deployment ──
rg4_name = "rg-lab-a2a-demo"
rg4_apic = "apic-a2a-v2vnj5h4ckvgy"
rg4_apim = "apim-a2a-wihxr7"
rg4_apim_gateway = "https://apim-a2a-wihxr7.azure-api.net"

# ── RG 1: Central API Center ──
rg1_name = "rg-lab-apic-central"
rg1_location = "swedencentral"
rg1_apic_name = "apic-std-swedencentral"
rg1_deployment_name = "central-apic"

utils.print_ok("Variables initialized")
print(f"  RG 1 (Central APIC):   {rg1_name} ({rg1_location})")
print(f"  RG 2 (Mixed APIs):     {rg2_name}")
print(f"  RG 3 (FastMCP):        {rg3_name} ({rg3_location})")
print(f"  RG 4 (A2A Agents):     {rg4_name}")

✅ Variables initialized ⌚ 13:28:52.448877 
  Resource Group: rg-lab-mcp-demo-1-1
  APIM SKU:       Basicv2
  Location:       uksouth


### 1️⃣ List APIs in RG 2 — existing distributed API Center

This API Center was deployed previously. It has a mix of:
- **REST APIs** converted to **MCP servers** via APIM policies
- **Remote 3rd-party MCP servers** (Atlassian, Sentry, PayPal, etc.) registered for discovery

In [8]:
# List all APIs in RG 2's API Center
output = utils.run(
    f'az apic api list -g {rg2_name} -n {rg2_apic} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in RG 2 API Center", "Failed to list APIs")
if output.success:
    print(output.text)

# Snapshot for later comparison
output = utils.run(f'az apic api list -g {rg2_name} -n {rg2_apic} -o json', "", "")
if output.success and output.json_data:
    rg2_apis = output.json_data
    rg2_rest = sum(1 for api in rg2_apis if api.get('kind') == 'rest')
    rg2_mcp  = sum(1 for api in rg2_apis if api.get('kind') == 'mcp')
    print(f"\n📊 RG 2 Total: {len(rg2_apis)} APIs — {rg2_rest} REST, {rg2_mcp} MCP")

⚙️ Running: az group show --name rg-lab-mcp-demo-1-1 
👉🏽 Resource group rg-lab-mcp-demo-1-1 does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name rg-lab-mcp-demo-1-1 --location uksouth --tags source=ai-gateway 
✅ Resource group 'rg-lab-mcp-demo-1-1' created ⌚ 13:29:18.524228 :6s]
⚙️ Running: az deployment group create --name mcp-demo-1 --resource-group rg-lab-mcp-demo-1-1 --template-file demo-initial.bicep --parameters params-demo.json 
✅ ✅ Deployment 'mcp-demo-1' succeeded ⌚ 13:32:18.478148 :59s]


### 2️⃣ Show APIM APIs in RG 2 — including MCP type APIs

In [9]:
# Use ARM REST API to see MCP-type APIs (az apim api list doesn't show 'type' field)
subscription_id = utils.get_current_subscription()

output = utils.run(
    f'az rest --method GET '
    f'--url "https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg2_name}'
    f'/providers/Microsoft.ApiManagement/service/{rg2_apim}/apis?api-version=2024-06-01-preview" '
    f'--query "value[].{{Name:name, DisplayName:properties.displayName, Type:properties.type, Path:properties.path}}" -o table',
    f"Listed APIs on APIM '{rg2_apim}'", "Failed")
if output.success:
    print(output.text)

⚙️ Running: az deployment group show --name mcp-demo-1 -g rg-lab-mcp-demo-1-1 
✅ Retrieved deployment outputs ⌚ 13:32:30.589432 :7s]
👉🏽 APIM Service Name: apim-mcp-demo-1
👉🏽 APIM Gateway URL: https://apim-mcp-demo-1.azure-api.net
👉🏽 API Center Name: apic-demo-1-xbilmxrmx74wu
👉🏽 API Center API Environment: api
👉🏽 API Center MCP Environment: mcp
👉🏽 API Key: ****5f57
👉🏽 Weather MCP: https://apim-mcp-demo-1.azure-api.net/weather-mcp/mcp
👉🏽 Catalog MCP: https://apim-mcp-demo-1.azure-api.net/catalog-mcp/mcp
👉🏽 Place Order MCP: https://apim-mcp-demo-1.azure-api.net/order-mcp/mcp


---
## Part 2 — Deploy FastMCP Containers (RG 3)
---

> Deploy 4 containerized FastMCP servers into a **separate resource group** — simulating a different team/subscription.
> Each server is: Container App → APIM MCP Proxy → API Center registration.

### 3️⃣ Deploy infrastructure + containers via Bicep

| Layer | Resources |
|-------|-----------|
| **Monitoring** | Log Analytics, Application Insights, MCP Dashboard |
| **API Gateway** | API Management (Basicv2) with MCP diagnostics |
| **API Governance** | API Center with MCP environment |
| **Containers** | ACR + ACA Environment + 4 Container Apps |
| **MCP Proxies** | 4× Streamable MCP APIs in APIM |
| **Discoverability** | 4× API Center registrations with VS Code install links |

> ⏱️ First deployment takes ~5-8 minutes (APIM provisioning). Subsequent runs are incremental.

In [10]:
# Create RG 3 and deploy infrastructure
utils.create_resource_group(rg3_name, rg3_location)

apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": "Basicv2" },
        "apimName": { "value": rg3_apim_name },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "apicLocation": { "value": rg3_location },
        "apicServiceNamePrefix": { "value": rg3_apic_prefix }
    }
}

with open('params-demo-containers.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {rg3_deployment_name} --resource-group {rg3_name} "
    f"--template-file demo-mcp-containers.bicep --parameters params-demo-containers.json",
    f"✅ RG 3 deployment '{rg3_deployment_name}' succeeded",
    f"❌ RG 3 deployment failed"
)

⚙️ Running: az apic api list -g rg-lab-mcp-demo-1-1 -n apic-demo-1-xbilmxrmx74wu --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in API Center ⌚ 13:32:43.700626 :6s]
Name              Title                Kind
----------------  -------------------  ------
sentry            Sentry               mcp
cloudflare        Cloudflare           mcp
atlassian         Atlassian            mcp
paypal            Paypal               mcp
linear            Linear               mcp
intercom          Intercom             mcp
square            Square               mcp
asana             Asana                mcp
plaid             Plaid                mcp
swagger-petstore  Swagger Petstore     rest
weather-api       Weather API          rest
order-api         Place Order API      rest
catalog-api       Product Catalog API  rest
order-mcp         Place Order MCP      mcp
weather-mcp       Weather MCP          mcp
catalog-mcp       Product Catalog MCP  mcp

⚙️ Running: az apic api li

### 4️⃣ Retrieve deployment outputs

In [12]:
output = utils.run(f"az deployment group show --name {rg3_deployment_name} -g {rg3_name}",
                   "Retrieved RG 3 deployment outputs", "Failed")

if output.success and output.json_data:
    rg3_apim_service  = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service')
    rg3_gateway_url   = utils.get_deployment_output(output, 'apimGatewayUrl', 'APIM Gateway')
    rg3_apic          = utils.get_deployment_output(output, 'apicServiceName', 'API Center')
    rg3_acr_name      = utils.get_deployment_output(output, 'acrName', 'ACR Name')
    rg3_acr_server    = utils.get_deployment_output(output, 'acrLoginServer', 'ACR Server')

    rg3_weather_url   = utils.get_deployment_output(output, 'weatherMcpUrl', 'Weather Container')
    rg3_catalog_url   = utils.get_deployment_output(output, 'catalogMcpUrl', 'Catalog Container')
    rg3_order_url     = utils.get_deployment_output(output, 'orderMcpUrl', 'Order Container')
    rg3_calculator_url = utils.get_deployment_output(output, 'calculatorMcpUrl', 'Calculator Container')

    rg3_app_insights  = utils.get_deployment_output(output, 'appInsightsName', 'App Insights')

    rg3_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    rg3_api_key = rg3_subscriptions[0].get("key")
    utils.print_info(f"API Key: ****{rg3_api_key[-4:]}")

🔍 Validation — Initial Deployment
--------------------------------------------------
✅ REST APIs: all 3 expected APIs found ✅ ⌚ 13:42:26.887064 
👉🏽   Additional REST APIs (from API Center defaults): {'swagger-petstore'}
✅ MCP Servers: all 3 expected servers found ✅ ⌚ 13:42:26.888075 
👉🏽   Additional MCP Servers (from API Center defaults): {'plaid', 'paypal', 'sentry', 'asana', 'linear', 'cloudflare', 'square', 'atlassian', 'intercom'}
✅ Calculator is NOT yet registered — ready for add-on demo ✅ ⌚ 13:42:26.888075 


### 5️⃣ Build and push container images to ACR

In [ ]:
containers = [
    ("weather-mcp",    "src/weather/container"),
    ("catalog-mcp",    "src/product-catalog/container"),
    ("order-mcp",      "src/place-order/container"),
    ("calculator-mcp", "src/calculator/container"),
]

for image_name, context_dir in containers:
    output = utils.run(
        f"az acr build --registry {rg3_acr_name} --image {image_name}:latest --file {context_dir}/Dockerfile {context_dir}",
        f"✅ Built {image_name}", f"❌ Failed to build {image_name}")
    if not output.success:
        break

### 6️⃣ Update container apps with built images

In [ ]:
for image_name, _ in containers:
    output = utils.run(
        f"az containerapp update --name {image_name} --resource-group {rg3_name} "
        f"--image {rg3_acr_server}/{image_name}:latest",
        f"✅ Updated {image_name}", f"❌ Failed to update {image_name}")
    if not output.success:
        break

---
## Part 3 — Test FastMCP Containers
---

### 🧪 Test all 4 containers — direct and through APIM

MCP Streamable HTTP protocol: POST JSON-RPC → SSE response stream

In [15]:
def call_mcp(endpoint, tool_name, arguments, label=""):
    """Call an MCP tool and return the parsed result."""
    request = {
        "method": "tools/call",
        "params": {"name": tool_name, "arguments": arguments},
        "jsonrpc": "2.0", "id": 1
    }
    try:
        response = requests.post(endpoint, json=request, stream=True, timeout=30,
                                 headers={"Content-Type": "application/json", "Accept": "text/event-stream"})
        if response.status_code == 200:
            for line in response.iter_lines(decode_unicode=True):
                if line and line.startswith("data:"):
                    data = json.loads(line[5:].strip())
                    if "result" in data and "content" in data["result"]:
                        text = data["result"]["content"][0].get("text", "")
                        try:
                            result = json.loads(text)
                        except json.JSONDecodeError:
                            result = text
                        utils.print_ok(f"  {label}: ✅")
                        return result
            utils.print_ok(f"  {label}: ✅ (no data in stream)")
            return None
        else:
            utils.print_error(f"  {label}: HTTP {response.status_code}")
            return None
    except Exception as e:
        utils.print_error(f"  {label}: {e}")
        return None
    finally:
        response.close()

def list_mcp_tools(endpoint, label=""):
    """List available tools on an MCP endpoint."""
    request = {"method": "tools/list", "params": {}, "jsonrpc": "2.0", "id": 1}
    try:
        response = requests.post(endpoint, json=request, stream=True, timeout=30,
                                 headers={"Content-Type": "application/json", "Accept": "text/event-stream"})
        if response.status_code == 200:
            for line in response.iter_lines(decode_unicode=True):
                if line and line.startswith("data:"):
                    data = json.loads(line[5:].strip())
                    if "result" in data and "tools" in data["result"]:
                        tools = [t["name"] for t in data["result"]["tools"]]
                        print(f"  {label}: {', '.join(tools)}")
                        return tools
        return []
    except Exception as e:
        utils.print_error(f"  {label}: {e}")
        return []
    finally:
        response.close()

utils.print_ok("MCP helper functions defined")

👉🏽 Calling Place Order MCP → PlaceOrder-invoke(sku='SKU-1234', quantity=2)...
⚙️ Running: az account get-access-token --resource "https://azure-api.net/authorization-manager" 
✅ Place Order MCP: HTTP 200 ✅ ⌚ 13:47:02.410192 
{
    "error": {
        "code": "DirectApiRequestHasMoreThanOneAuthorization",
        "message": "The request has SAS authentication scheme and an additional authorization scheme or internal token scheme. Only one scheme should be used."
    }
}


In [ ]:
# Test each container — direct call
print("🧪 Direct container tests")
print("=" * 50)

tests = [
    (f"{rg3_weather_url}/weather/mcp",      "get_weather",       {"city": "London"},             "Weather"),
    (f"{rg3_catalog_url}/catalog/mcp",       "search_products",   {"query": "laptop"},            "Catalog"),
    (f"{rg3_order_url}/order/mcp",           "place_order",       {"product_id": "P001", "quantity": 1}, "Order"),
    (f"{rg3_calculator_url}/calculator/mcp", "calculate",         {"operation": "multiply", "a": 7, "b": 8}, "Calculator"),
]

results = {}
for endpoint, tool, args, label in tests:
    result = call_mcp(endpoint, tool, args, f"{label} (direct)")
    results[f"{label}_direct"] = result is not None
    if result:
        print(f"    → {json.dumps(result, indent=2)[:200]}")

# Test through APIM
print(f"\n🧪 APIM proxy tests ({rg3_gateway_url})")
print("=" * 50)

apim_tests = [
    (f"{rg3_gateway_url}/weather-mcp/mcp",    "get_weather",       {"city": "Paris"},              "Weather"),
    (f"{rg3_gateway_url}/catalog-mcp/mcp",     "search_products",   {"query": "phone"},             "Catalog"),
    (f"{rg3_gateway_url}/order-mcp/mcp",       "place_order",       {"product_id": "P002", "quantity": 3}, "Order"),
    (f"{rg3_gateway_url}/calculator-mcp/mcp",  "calculate",         {"operation": "add", "a": 42, "b": 17}, "Calculator"),
]

for endpoint, tool, args, label in apim_tests:
    result = call_mcp(endpoint, tool, args, f"{label} (APIM)")
    results[f"{label}_apim"] = result is not None

passed = sum(1 for v in results.values() if v)
print(f"\n📊 Results: {passed}/{len(results)} tests passed")

### 7️⃣ Verify RG 3's local API Center

Each container MCP server was auto-registered in RG 3's own API Center during Bicep deployment.

In [16]:
output = utils.run(
    f'az apic api list -g {rg3_name} -n {rg3_apic} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed APIs in RG 3 API Center", "Failed")
if output.success:
    print(output.text)

# Snapshot
output = utils.run(f'az apic api list -g {rg3_name} -n {rg3_apic} -o json', "", "")
if output.success and output.json_data:
    rg3_apis = output.json_data
    rg3_mcp = sum(1 for api in rg3_apis if api.get('kind') == 'mcp')
    print(f"\n📊 RG 3 Total: {len(rg3_apis)} APIs — {rg3_mcp} MCP Servers")

⚙️ Running: az deployment group create --name mcp-demo-1-calculator --resource-group rg-lab-mcp-demo-1-1 --template-file demo-add-calculator.bicep --parameters params-demo-calculator.json 
✅ ✅ Calculator MCP deployed! ⌚ 13:48:09.539286 :50s]


---
## Part 4 — Deploy Central API Center (RG 1)
---

> 💡 **This is the key architecture moment.**  
> The central API Center acts as the **single pane of glass** for all APIs and MCP servers across the organization — regardless of which resource group, subscription, or APIM instance hosts them.

In [17]:
# Create RG 1 and deploy central API Center
utils.create_resource_group(rg1_name, rg1_location)

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apicServiceName": { "value": rg1_apic_name },
        "location": { "value": rg1_location }
    }
}

with open('params-central-apic.json', 'w') as f:
    f.write(json.dumps(bicep_parameters))

output = utils.run(
    f"az deployment group create --name {rg1_deployment_name} --resource-group {rg1_name} "
    f"--template-file demo-central-apic.bicep --parameters params-central-apic.json",
    f"✅ Central API Center deployed in {rg1_location}",
    f"❌ Central API Center deployment failed"
)

if output.success and output.json_data:
    rg1_apic_actual = utils.get_deployment_output(output, 'name', 'Central APIC Name')
    utils.print_info(f"Central APIC: {rg1_apic_actual}")

⚙️ Running: az apic api list -g rg-lab-mcp-demo-1-1 -n apic-demo-1-xbilmxrmx74wu --query "[].{Name:name, Title:title, Kind:kind}" -o table 
✅ Listed APIs in API Center ⌚ 13:48:21.693036 :5s]
Name              Title                Kind
----------------  -------------------  ------
sentry            Sentry               mcp
cloudflare        Cloudflare           mcp
atlassian         Atlassian            mcp
paypal            Paypal               mcp
linear            Linear               mcp
intercom          Intercom             mcp
square            Square               mcp
asana             Asana                mcp
plaid             Plaid                mcp
swagger-petstore  Swagger Petstore     rest
weather-api       Weather API          rest
order-api         Place Order API      rest
catalog-api       Product Catalog API  rest
order-mcp         Place Order MCP      mcp
weather-mcp       Weather MCP          mcp
catalog-mcp       Product Catalog MCP  mcp
calculator-api    Calculato

---
## Part 5 — Aggregate APIs into Central Catalog
---

> Pull APIs from **both distributed API Centers** into the central catalog.  
> This simulates a platform team aggregating APIs from multiple product teams.

In [ ]:
def get_apic_apis(resource_group, apic_name):
    """Fetch all APIs from an API Center instance."""
    output = utils.run(f'az apic api list -g {resource_group} -n {apic_name} -o json', "", "")
    if output.success and output.json_data:
        return output.json_data
    return []

def get_apic_deployments(resource_group, apic_name, api_name):
    """Fetch deployments for an API to get the runtimeUri."""
    output = utils.run(
        f'az apic api deployment list -g {resource_group} -n {apic_name} --api-id {api_name} -o json', "", "")
    if output.success and output.json_data:
        return output.json_data
    return []

def register_api_in_central(api, source_rg, source_apic, source_label):
    """Register a single API from a distributed APIC into the central APIC."""
    api_name = api['name']
    kind = api.get('kind', 'rest')
    title = api.get('title', api_name)
    description = api.get('description', '') or api.get('summary', '') or f'{title} from {source_label}'
    # Map kind to the correct central APIC environment
    env_map = {'mcp': 'mcp', 'a2a': 'a2a', 'rest': 'api'}
    env_name = env_map.get(kind, 'api')

    # Register the API
    cmd = (f'az apic api create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} '
           f'--title "{title}" --kind {kind} '
           f'--description "{description[:200]}" ')
    utils.run(cmd, "", "")

    # Add version
    utils.run(f'az apic api version create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} '
              f'--version-id 1-0-0 --title "1.0.0" --lifecycle-stage production', "", "")

    # Add definition
    utils.run(f'az apic api definition create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} '
              f'--version-id 1-0-0 --definition-id default --title "Default"', "", "")

    # Get runtimeUri from source deployments
    deployments = get_apic_deployments(source_rg, source_apic, api_name)
    runtime_uris = []
    for dep in deployments:
        uris = dep.get('server', {}).get('runtimeUri', [])
        runtime_uris.extend(uris)

    # Register deployment with runtimeUri pointing to the source APIM
    runtime_json = json.dumps(runtime_uris) if runtime_uris else '[]'
    server_block = f'{{"runtimeUri": {runtime_json}}}'
    utils.run(
        f"az apic api deployment create -g {rg1_name} -n {rg1_apic_name} --api-id {api_name} "
        f"--deployment-id {api_name}-from-{source_label} --title \"{title} ({source_label})\" "
        f"--environment-id /workspaces/default/environments/{env_name} "
        f"--definition-id /workspaces/default/apis/{api_name}/versions/1-0-0/definitions/default "
        f"--server \"{server_block}\" ",
        "", "")

    return runtime_uris

utils.print_ok("Aggregation functions defined (supports REST, MCP, and A2A)")

🔍 Validation — Before vs After Comparison
                           BEFORE      AFTER       DIFF
  -------------------- ---------- ---------- ----------
  REST APIs                     4          5         +1
  MCP Servers                  12         13         +1
  Total                        16         18         +2

  🆕 Newly discovered APIs:
     • calculator-api (rest)
     • calculator-mcp (mcp)

✅ Calculator API + MCP auto-discovered in API Center ✅ ⌚ 13:49:55.992104 
✅ REST API count increased by 1 (4 → 5) ✅ ⌚ 13:49:55.992104 
✅ MCP Server count increased by 1 (12 → 13) ✅ ⌚ 13:49:55.992104 

✅ 🎉 Auto-discovery validated — new MCP servers appear automatically! ⌚ 13:49:55.992104 


### 8️⃣ Sync APIs from RG 2 (mixed REST→MCP) → Central APIC

In [ ]:
# Aggregate from RG 2 (mixed REST→MCP APIs)
rg2_apis = get_apic_apis(rg2_name, rg2_apic)
print(f"📥 Syncing {len(rg2_apis)} APIs from RG 2 ({rg2_apic}) → Central APIC")
print("=" * 60)

rg2_synced = 0
for api in rg2_apis:
    uris = register_api_in_central(api, rg2_name, rg2_apic, "rg2")
    uri_str = uris[0] if uris else "—"
    print(f"  ✅ {api['name']:25} ({api.get('kind','?'):4}) → {uri_str}")
    rg2_synced += 1

print(f"\n📊 Synced {rg2_synced} APIs from RG 2")

### 9️⃣ Sync APIs from RG 3 (FastMCP containers) → Central APIC

In [ ]:
# Aggregate from RG 3 (FastMCP containers)
rg3_apis = get_apic_apis(rg3_name, rg3_apic)
print(f"📥 Syncing {len(rg3_apis)} APIs from RG 3 ({rg3_apic}) → Central APIC")
print("=" * 60)

rg3_synced = 0
for api in rg3_apis:
    uris = register_api_in_central(api, rg3_name, rg3_apic, "rg3")
    uri_str = uris[0] if uris else "—"
    print(f"  ✅ {api['name']:25} ({api.get('kind','?'):4}) → {uri_str}")
    rg3_synced += 1

print(f"\n📊 Synced {rg3_synced} APIs from RG 3")

---
## Part 6 — Unified Discovery from Central API Center
---

> 💡 **Single pane of glass** — query one API Center, discover APIs from all environments.

In [ ]:
# List everything in the central API Center
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[].{{Name:name, Title:title, Kind:kind}}" -o table',
    "Listed ALL APIs in Central API Center", "Failed")
if output.success:
    print(output.text)

output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} -o json', "", "")
if output.success and output.json_data:
    central_apis = output.json_data
    central_rest = sum(1 for api in central_apis if api.get('kind') == 'rest')
    central_mcp  = sum(1 for api in central_apis if api.get('kind') == 'mcp')
    print(f"\n📊 Central Catalog: {len(central_apis)} APIs — {central_rest} REST, {central_mcp} MCP")
    print(f"   Aggregated from: RG 2 ({rg2_synced} APIs) + RG 3 ({rg3_synced} APIs)")

### 🔍 Filter: MCP servers with deployment endpoints from Central Catalog

In [ ]:
# Show MCP servers from central catalog with their deployment endpoints
output = utils.run(
    f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'mcp\'].{{Name:name, Title:title, Kind:kind}}" -o table',
    "MCP servers in Central Catalog", "Failed")
if output.success:
    print(output.text)

# Show deployments with runtime URIs for each MCP server
print("\n🔗 MCP Server Endpoints (from Central Catalog deployments)")
print("=" * 80)
mcp_endpoints = {}
for api in central_apis:
    if api.get('kind') == 'mcp':
        deployments = get_apic_deployments(rg1_name, rg1_apic_name, api['name'])
        for dep in deployments:
            uris = dep.get('server', {}).get('runtimeUri', [])
            source = dep.get('title', '')
            for uri in uris:
                print(f"  {api['name']:25} → {uri}  ({source})")
                mcp_endpoints[api['name']] = uri

print(f"\n📊 {len(mcp_endpoints)} MCP servers discoverable with invocation endpoints")

### 🔌 Dynamic connection — discover and invoke MCP servers from Central Catalog

An agent or client can:
1. Query the central API Center for `kind == 'mcp'`
2. Get the `runtimeUri` from deployments
3. Connect and invoke tools — **without knowing which APIM or RG hosts them**

In [19]:
# Dynamically discover and invoke MCP servers from the central catalog
print("🔌 Dynamic MCP Discovery + Invocation from Central Catalog")
print("=" * 65)

# Step 1: Discover MCP servers
output = utils.run(f'az apic api list -g {rg1_name} -n {rg1_apic_name} --query "[?kind==\'mcp\']" -o json', "", "")
discovered_servers = output.json_data if output.success and output.json_data else []
print(f"\n  Step 1: Discovered {len(discovered_servers)} MCP servers in central catalog")

# Step 2: Get runtime URIs
print(f"  Step 2: Resolving runtime endpoints...")
invocable = {}
for api in discovered_servers:
    deps = get_apic_deployments(rg1_name, rg1_apic_name, api['name'])
    for dep in deps:
        for uri in dep.get('server', {}).get('runtimeUri', []):
            # Build MCP endpoint from the runtime URI
            mcp_url = f"{uri}/mcp" if not uri.endswith('/mcp') else uri
            invocable[api['name']] = mcp_url
            break  # first deployment

print(f"  Step 3: Invoking tools on discovered servers...\n")

# Step 3: Call tools/list on each discovered server
for name, endpoint in sorted(invocable.items()):
    tools = list_mcp_tools(endpoint, label=name)
    if not tools:
        print(f"  ⚠️  {name}: no tools returned (may need auth or server not running)")

print(f"\n📊 Connected to {len(invocable)} MCP servers via central catalog")

NameError: name 'calculator_mcp_endpoint' is not defined

---
## Part 7 — Agent Workflow: Cross-Environment Tool Chaining
---

> An agent discovers MCP servers from the **central catalog** and chains tools across **multiple APIM instances** — Weather from RG 2, Calculator from RG 3, etc.

### Simulated agent scenario:
1. **Discover** available MCP servers from central API Center
2. **Search catalog** (RG 2's APIM) → find a laptop
3. **Place order** (RG 2's APIM) → order it
4. **Calculate total** (RG 3's APIM) → compute with tax
5. **Check weather** (RG 3's APIM) → for delivery planning

In [ ]:
print("🤖 Agent Workflow — Cross-Environment Tool Chaining")
print("=" * 65)

# Step 1: Agent discovers MCP servers from central catalog
print("\n📋 Step 1: Agent queries central API Center for MCP servers")
mcp_servers = {api['name']: api.get('title','') for api in discovered_servers}
for name, title in sorted(mcp_servers.items()):
    endpoint = invocable.get(name, '—')
    source = "RG 2" if "rg2" in endpoint or rg2_apim in endpoint else "RG 3" if rg3_apim_name in endpoint else "?"
    print(f"  • {title:30} [{source}] → {endpoint[:60]}...")

# Step 2: Search catalog (uses RG 2 or RG 3 depending on what's available)
print("\n🔍 Step 2: Agent calls catalog MCP → search for 'laptop'")
catalog_ep = invocable.get('catalog-mcp')
if catalog_ep:
    result = call_mcp(catalog_ep, "search_products", {"query": "laptop"}, "Catalog search")
    if result:
        print(f"    → Found: {json.dumps(result)[:200]}")

# Step 3: Place order
print("\n📦 Step 3: Agent calls order MCP → place order")
order_ep = invocable.get('order-mcp')
if order_ep:
    result = call_mcp(order_ep, "place_order", {"product_id": "LAPTOP-001", "quantity": 2}, "Place order")
    if result:
        print(f"    → Order: {json.dumps(result)[:200]}")

# Step 4: Calculate total with tax
print("\n🧮 Step 4: Agent calls calculator MCP → compute total with tax")
calc_ep = invocable.get('calculator-mcp')
if calc_ep:
    result = call_mcp(calc_ep, "calculate", {"operation": "multiply", "a": 999.99, "b": 1.08}, "Calculate total")
    if result:
        print(f"    → Total with 8% tax: ${result.get('result', 'N/A')}")

# Step 5: Check weather for delivery
print("\n🌤️ Step 5: Agent calls weather MCP → check delivery weather")
weather_ep = invocable.get('weather-mcp')
if weather_ep:
    result = call_mcp(weather_ep, "get_weather", {"city": "Seattle"}, "Weather check")
    if result:
        print(f"    → Weather: {json.dumps(result)[:200]}")

print("\n" + "=" * 65)
print("✅ Agent workflow complete — tools chained across RG 2 + RG 3 via central catalog!")

---
## ✅ Demo Summary
---

In [ ]:
print("=" * 70)
print("  🎬  FEDERATED API GOVERNANCE DEMO — SUMMARY")
print("=" * 70)
print()
print("  Architecture:")
print(f"    RG 1 — Central APIC:   {rg1_apic_name} ({rg1_location})")
print(f"    RG 2 — Mixed APIs:     {rg2_apic} ({rg2_name})")
print(f"    RG 3 — FastMCP:        {rg3_apic} ({rg3_name})")
print()
print("  API Inventory:")
print(f"    RG 2: {len(rg2_apis)} APIs ({rg2_rest} REST, {rg2_mcp} MCP)")
print(f"    RG 3: {len(rg3_apis)} APIs ({rg3_mcp} MCP)")
print(f"    Central: {len(central_apis)} APIs aggregated ({central_rest} REST, {central_mcp} MCP)")
print()
print("  Key Takeaways:")
print("    • Central API Center = single pane of glass for all API/MCP discovery")
print("    • APIM = invocation layer — each RG has its own APIM instance")
print("    • Agents discover from Central APIC, invoke through distributed APIMs")
print("    • New APIs auto-registered in local APIC, synced to central on demand")
print("    • Works across subscriptions, resource groups, and regions")
print("=" * 70)

---
## 🗑️ Clean up resources

Uncomment and run to delete **only the new resource groups** created by this demo.  
RG 2 (`rg-lab-mcp-demo-1-1`) is left intact as it was pre-existing.

In [ ]:
# Uncomment to delete demo resources:

# Delete RG 3 (FastMCP containers)
# utils.run(f"az group delete --name {rg3_name} --yes --no-wait",
#           f"RG 3 '{rg3_name}' deletion initiated", "Failed")

# Delete RG 1 (Central APIC)
# utils.run(f"az group delete --name {rg1_name} --yes --no-wait",
#           f"RG 1 '{rg1_name}' deletion initiated", "Failed")

# Note: RG 2 (rg-lab-mcp-demo-1-1) is NOT deleted — it was pre-existing
print("⚠️  Uncomment the lines above to delete demo resources")